# Fixed: Rule-based Gujarati Morphological Segmentation
**What this notebook does (high-success, common-suffix rules):**

- Loads local `.txt` files from a folder you specify
- Normalizes and tokenizes Gujarati text
- Applies a compact list of **common Gujarati suffixes** (locative, instrumental, plural, case markers) to split **stem + suffix**
- Shows demo outputs and corpus samples

**How to use:** Edit the `corpus_dir` variable in the "Load corpus" cell to point at your local folder containing `.txt` files and run all cells.

Generated on: 2025-10-29T18:57:19.534827


In [1]:
import os
import unicodedata
import pandas as pd
from collections import Counter

def normalize_text(s):
    return unicodedata.normalize('NFC', s).strip()


In [2]:
# --- USER: set this to your local path ---
corpus_dir = r"X:/DJ Sanghvi/sem 7/nlp/NLP_LAB_GYANGUJ/data/next"
# ----------------------------------------

texts = []
for fn in os.listdir(corpus_dir):
    if fn.endswith('.txt'):
        path = os.path.join(corpus_dir, fn)
        with open(path, 'r', encoding='utf-8') as f:
            txt = f.read().strip()
            if txt:
                texts.append((fn, normalize_text(txt)))

if not texts:
    print('No .txt files found in', corpus_dir)
else:
    print(f'Loaded {len(texts)} files from', corpus_dir)

df_files = pd.DataFrame(texts, columns=['File', 'Content'])
df_files.head(3)


Loaded 8 files from X:/DJ Sanghvi/sem 7/nlp/NLP_LAB_GYANGUJ/data/next


,File,Content
0,class11_biology_prepro.txt,જીવવિજ્ઞાન ધોરણ ઉઠે પ્રતિજ્ઞાપત્ર ભારત મારો દે...
1,class11_chemistry_prepro.txt,પરમાણુનો ક્વોન્ટમ યાંત્રિકીય નમૂનો તત્ત્વોનું ...
2,class11_maths_prepro.txt,પ્રકરણમાં આપણે ગણ સંબંધિત પાયાની વ્યાખ્યાઓ ગણ ...


In [3]:
words = []
for fn, txt in texts:
    tokens = [t for t in txt.split() if t]
    words.extend(tokens)

print('Total tokens (approx):', len(words))
sample_words = words[:2000]
df = pd.DataFrame({'Word': sample_words})
df.head(10)


Total tokens (approx): 507730


,Word
0,જીવવિજ્ઞાન
1,ધોરણ
2,ઉઠે
3,પ્રતિજ્ઞાપત્ર
4,ભારત
5,મારો
6,દેશ
7,બધાં
8,ભારતીયો
9,મારા


In [4]:
common_suffixes = [
    'માં','માંથી','ઓમાં','ઓને','ઓના','ઓથી',
    'થી','ને','ના','માં','નું','ની','ઓ','ે','ો'
]
common_suffixes = sorted(set([normalize_text(s) for s in common_suffixes if s.strip()]), key=len, reverse=True)

def extract_stem_rule(word, suffix_list=common_suffixes):
    w = normalize_text(word)
    for suf in suffix_list:
        if len(suf) == 0:
            continue
        if w.endswith(suf):
            stem = w[:-len(suf)]
            if stem == '':
                return w, suf
            return stem, suf
    return w, ''

examples = ['શિક્ષકો', 'સરકારમાં', 'મિત્રોથી', 'કલાકો', 'શેરબજારો']
for ex in examples:
    stem, suf = extract_stem_rule(ex)
    print(f"{ex} -> stem:'{stem}'  suffix:'{suf}'")


શિક્ષકો -> stem:'શિક્ષક'  suffix:'ો'
સરકારમાં -> stem:'સરકાર'  suffix:'માં'
મિત્રોથી -> stem:'મિત્રો'  suffix:'થી'
કલાકો -> stem:'કલાક'  suffix:'ો'
શેરબજારો -> stem:'શેરબજાર'  suffix:'ો'


In [5]:
N = 200
results = []
for w in sample_words[:N]:
    stem, suf = extract_stem_rule(w)
    results.append((w, stem, suf))

df_seg = pd.DataFrame(results, columns=['Word', 'Stem', 'Suffix'])
df_seg.head(30)

non_empty = df_seg['Suffix'].apply(lambda x: bool(x)).sum()
print(f"Of first {N} tokens, {non_empty} ({non_empty/N:.2%}) had a matched suffix from the common list")

print('\nTop suffixes in sample:')
print(df_seg['Suffix'].value_counts().head(20))


Of first 200 tokens, 41 (20.50%) had a matched suffix from the common list

Top suffixes in sample:
Suffix
       159
ો       10
ને      10
ે        8
ના       6
માં      4
ની       2
થી       1
Name: count, dtype: int64


In [6]:
demo_words = [
    'શિક્ષકો','સરકારે','મિત્રોથી','કલાકો','શેરબજારો',
    'વિદ્યાર્થીઓ','ગામોમાં','શાળામાં','અધ્યાપકો','વિજ્ઞાનીઓ',
    'પુસ્તકોમાં','પત્રકારો','રોગોથી','કામદારો','શિક્ષણમાં',
    'વૃક્ષો','બાળકોએ','રમતોમાં','મિત્રતામાં','વિદ્યાપીઠો',
    'પ્રયત્નોથી','સમયોએ','નાગરિકોએ','આશાઓથી','વિચારધારાઓ'
]

for w in demo_words:
    stem, suf = extract_stem_rule(w)
    print(f"{w:15s} -> stem: '{stem:12s}' | suffix: '{suf}'")


શિક્ષકો         -> stem: 'શિક્ષક      ' | suffix: 'ો'
સરકારે          -> stem: 'સરકાર       ' | suffix: 'ે'
મિત્રોથી        -> stem: 'મિત્રો      ' | suffix: 'થી'
કલાકો           -> stem: 'કલાક        ' | suffix: 'ો'
શેરબજારો        -> stem: 'શેરબજાર     ' | suffix: 'ો'
વિદ્યાર્થીઓ     -> stem: 'વિદ્યાર્થી  ' | suffix: 'ઓ'
ગામોમાં         -> stem: 'ગામો        ' | suffix: 'માં'
શાળામાં         -> stem: 'શાળા        ' | suffix: 'માં'
અધ્યાપકો        -> stem: 'અધ્યાપક     ' | suffix: 'ો'
વિજ્ઞાનીઓ       -> stem: 'વિજ્ઞાની    ' | suffix: 'ઓ'
પુસ્તકોમાં      -> stem: 'પુસ્તકો     ' | suffix: 'માં'
પત્રકારો        -> stem: 'પત્રકાર     ' | suffix: 'ો'
રોગોથી          -> stem: 'રોગો        ' | suffix: 'થી'
કામદારો         -> stem: 'કામદાર      ' | suffix: 'ો'
શિક્ષણમાં       -> stem: 'શિક્ષણ      ' | suffix: 'માં'
વૃક્ષો          -> stem: 'વૃક્ષ       ' | suffix: 'ો'
બાળકોએ          -> stem: 'બાળકોએ      ' | suffix: ''
રમતોમાં         -> stem: 'રમતો        ' | suffix: 'માં'
મિત્રતામાં      -

In [7]:
def extract_stem_from_label_compat(word, label=None):
    try:
        if label and '1' in str(label):
            lab = str(label)
            pos = lab.index('1')
            stem = word[:pos+1]
            suffix = word[pos+1:]
            return stem, suffix
    except Exception:
        pass
    return extract_stem_rule(word)

print('\nCompatibility demos:')
print(extract_stem_from_label_compat('કલાકો', '00010'))
print(extract_stem_from_label_compat('કલાકો', '00000'))



Compatibility demos:
('કલાક', 'ો')
('કલાક', 'ો')


In [ ]:
out_csv = 'temp6.csv'
df_seg.to_csv(out_csv, index=False)
print('Saved segmented sample to', out_csv)


Saved segmented sample to temp.csv
